# Overview

- transformers の動作確認。
- [Hugging Face Transformers - Get started !](https://mats2.net/rakutenpovo2-dual)

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BertTokenizer

torch.set_grad_enabled(False)

In [2]:
# Store the model we want to use
MODEL_NAME = "bert-base-cased"

# We need to create the model and tokenizer
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

上記の2行のコードだけで、BERTの事前学習済みモデルを使用する準備が整いました。トークナイザーは、生のテキスト入力を、モデルが操作できる方法でテキスト入力を表す整数のシーケンスにマッピングすることを可能にします。

In [3]:
# Tokens comes from a process that splits the input into sub-entities with interesting linguistic properties. 
tokens = tokenizer.tokenize("This is an input example")
print("Tokens: {}".format(tokens))

# This is not sufficient for the model, as it requires integers as input, 
# not a problem, let's convert tokens to ids.
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Tokens id: {}".format(tokens_ids))

# Add the required special tokens
tokens_ids = tokenizer.build_inputs_with_special_tokens(tokens_ids)

# We need to convert to a Deep Learning framework specific format, let's use PyTorch for now.
tokens_pt = torch.tensor([tokens_ids])
print("Tokens PyTorch: {}".format(tokens_pt))

# Now we're ready to go through BERT with out input
outputs, pooled = model(tokens_pt, return_dict=False)
print("Tokenwise output: {}, Pooled output: {}".format(outputs.shape, pooled.shape))

Tokens: ['This', 'is', 'an', 'input', 'example']
Tokens id: [1188, 1110, 1126, 7758, 1859]
Tokens PyTorch: tensor([[ 101, 1188, 1110, 1126, 7758, 1859,  102]])
Tokenwise output: torch.Size([1, 7, 768]), Pooled output: torch.Size([1, 768])


ご覧のように、BERTは2つのテンソルを出力する。

- 一つは入力の各トークンに対して生成された表現（1, NB_TOKENS, REPRESENTATION_SIZE）である。
- 1 つは入力全体の表現を集約したもの (1, REPRESENTATION_SIZE)

最初のトークンベースの表現は、タスクでシーケンス表現を維持する必要があり、トークンレベルで操作したい場合に利用できる。これは特に名前付き固有表現認識や質問応答で有効である。

2つ目の集約型表現は、シーケンスの全体的な文脈を抽出する必要があり、細かいトークンレベルを必要としない場合に特に有効である。これはシーケンスのSentiment-AnalysisやInformation Retrievalの場合である。

前のセクションで見たコードは、単純なモデル呼び出しに必要なすべてのステップを紹介しました。より日常的な使い方のために、トランスフォーマーはより高レベルのメソッドを提供し、あなたの NLP の旅をより簡単にしてくれます。 前の例を改良してみましょう

In [4]:
# tokens = tokenizer.tokenize("This is an input example")
# tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
# tokens_pt = torch.tensor([tokens_ids])

# This code can be factored into one-line as follow
tokens_pt2 = tokenizer.encode_plus("This is an input example", return_tensors="pt")

for key, value in tokens_pt2.items():
    print("{}:\n\t{}".format(key, value))

outputs2, pooled2 = model(**tokens_pt2, return_dict=False)
print("Difference with previous code: ({}, {})".format((outputs2 - outputs).sum(), (pooled2 - pooled).sum()))

input_ids:
	tensor([[ 101, 1188, 1110, 1126, 7758, 1859,  102]])
token_type_ids:
	tensor([[0, 0, 0, 0, 0, 0, 0]])
attention_mask:
	tensor([[1, 1, 1, 1, 1, 1, 1]])
Difference with previous code: (0.0, 0.0)


上で見たように、encode_plusメソッドは、モデルを通過するすべての必要なパラメータを生成する便利な方法を提供します。

さらに、いくつかの追加のテンソルを生成していることに気づかれたかもしれません。

- token_type_ids。このテンソルはすべてのトークンを対応するセグメントにマッピングします（下記参照）。
- attention_mask。このテンソルは異なる長さのシーケンスでパディングされた値を「マスク」するために使われる（下記参照）。

In [5]:
# Single segment input
single_seg_input = tokenizer.encode_plus("This is a sample input")

# Multiple segment input
multi_seg_input = tokenizer.encode_plus("This is segment A", "This is segment B")

print("Single segment token (str): {}".format(tokenizer.convert_ids_to_tokens(single_seg_input['input_ids'])))
print("Single segment token (int): {}".format(single_seg_input['input_ids']))
print("Single segment type       : {}".format(single_seg_input['token_type_ids']))

# Segments are concatened in the input to the model, with 
print()
print("Multi segment token (str): {}".format(tokenizer.convert_ids_to_tokens(multi_seg_input['input_ids'])))
print("Multi segment token (int): {}".format(multi_seg_input['input_ids']))
print("Multi segment type       : {}".format(multi_seg_input['token_type_ids']))

Single segment token (str): ['[CLS]', 'This', 'is', 'a', 'sample', 'input', '[SEP]']
Single segment token (int): [101, 1188, 1110, 170, 6876, 7758, 102]
Single segment type       : [0, 0, 0, 0, 0, 0, 0]

Multi segment token (str): ['[CLS]', 'This', 'is', 'segment', 'A', '[SEP]', 'This', 'is', 'segment', 'B', '[SEP]']
Multi segment token (int): [101, 1188, 1110, 6441, 138, 102, 1188, 1110, 6441, 139, 102]
Multi segment type       : [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


**フレームワークの相互運用性**  
トランスフォーマーの最も強力な機能の1つは、ユーザーが苦痛を感じることなく、PyTorchからTensorflowにシームレスに移行できることです。

TensorFlowで学習したウェイトをPyTorchのモデルにロードしたり、逆にTensorFlowで学習したウェイトをPyTorchのモデルにロードする便利なメソッドをいくつか提供します。

In [6]:
from transformers import TFBertModel, BertModel

# Let's load a BERT model for TensorFlow and PyTorch
model_tf = TFBertModel.from_pretrained('bert-base-cased')
model_pt = BertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'
  

In [7]:
# transformers generates a ready to use dictionary with all the required parameters for the specific framework.
input_tf = tokenizer.encode_plus("This is a sample input", return_tensors="tf")
input_pt = tokenizer.encode_plus("This is a sample input", return_tensors="pt")

# Let's compare the outputs
output_tf, output_pt = model_tf(input_tf, return_dict=False), model_pt(**input_pt, return_dict=False)

# Models outputs 2 values (The value for each tokens, the pooled representation of the input sentence)
# Here we compare the output differences between PyTorch and TensorFlow.
for name, o_tf, o_pt in zip(["output", "pooled"], output_tf, output_pt):
    print("{} differences: {}".format(name, (o_tf.numpy() - o_pt.numpy()).sum()))

output differences: -2.74624653684441e-07
pooled differences: -6.137881428003311e-06


もっと軽くしたい？もっと軽くしたいですか？もっと軽く、もっと早く？
Transformerベースのモデルを使用する際の主な懸念事項の1つは、それらが必要とする計算能力です。このノートブックでは、一般的なマシンで実行できるBERTモデルを使用していますが、すべてのモデルがそうであるとは限りません。

例えば、Googleは数ヶ月前にT5をリリースしましたトランスフォーマーに基づくエンコーダ/デコーダアーキテクチャと110億以上のパラメータを持つトランスフォーマーで利用可能です。マイクロソフトも最近、170億のパラメータを使用するTuring-NLGでゲームに参加しました。このようなモデルは、重みの保存に数十ギガバイト、実行には膨大な計算機インフラが必要で、一般人には到底無理な話です

トランスフォーマーパラメータ

TransformerベースのNLPを誰でも使えるようにすることを目標に、私たち@huggingfaceはDistillationと呼ばれる学習プロセスを利用したモデルを開発し、そのようなモデルの実行に必要なリソースを大幅に減らし、パフォーマンスの低下をほぼゼロにすることを可能にしました。

Distillationのプロセス全体については、このノートブックの範囲外ですが、このテーマについてより多くの情報が必要な場合は、DistilBERT論文の著者である私の同僚Victor SANHが書いたこのMediumの記事を参照してください、また、直接論文（Sanh & al., 2019）を見るのもよいでしょう。

もちろん、トランスフォーマーでは、いくつかのモデルを抽出して、ライブラリで直接利用できるようにしています !

In [8]:
from transformers import DistilBertModel

bert_distil = DistilBertModel.from_pretrained('distilbert-base-cased')
input_pt = tokenizer.encode_plus(
    'This is a sample input to demonstrate performance of distiled models especially inference time', 
    return_tensors="pt"
)


%time _ = bert_distil(input_pt['input_ids'])
%time _ = model_pt(input_pt['input_ids'])

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CPU times: user 67.3 ms, sys: 2.78 ms, total: 70.1 ms
Wall time: 36.9 ms
CPU times: user 134 ms, sys: 880 µs, total: 135 ms
Wall time: 67.5 ms


**コミュニティ提供モデル**  
最後になりましたが、このノートブックの前半で、NLPコミュニティが学習済みモデルを交換するためのリポジトリとして、Hugging Faceトランスフォーマーを紹介しました。私たちはこの機能とエンドユーザーに提供するすべての可能性を強調したかったのです。

コミュニティの事前学習済みモデルを利用するには、組織名とモデルの名前をfrom_pretrainedに提供するだけです。

現在、コミュニティから提供された50以上のモデルがあり、さらに毎日追加されています。

In [9]:
# バイエルン州立図書館からドイツ語 BERT を読み込もう
de_bert = BertModel.from_pretrained("dbmdz/bert-base-german-cased")
de_tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-german-cased")

de_input = de_tokenizer.encode_plus(
    "Hugging Face ist einen französische Firma Mitarbeitern in New-York.", 
    return_tensors="pt"
)
output_de, pooled_de = de_bert(**de_input, return_dict=False)

Downloading:   0%|          | 0.00/456 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/234k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]